In [1]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [44.8 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,131 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,009 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,400 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.co

In [2]:
!apt-get -y install nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libaccinj64-11.5 libatk-wrapper-java libatk-wrapper-java-jni
  libbabeltrace1 libcub-dev libcublas11 libcublaslt11 libcudart11.0 libcufft10 libcufftw10
  libcuinj64-11.5 libcupti-dev libcupti-doc libcupti11.5 libcurand10 libcusolver11 libcusolvermg11
  libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libfontenc1 libgail-common libgail18
  libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglvnd-core-dev libglvnd-dev libglx-dev
  libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libipt2 libnppc11 libnppial11 libnppicc11 libnppidei11
  libnppif11 libnppig11 libnppim11 libnppist11 libnppisu11 libnppitc11 libnpps11 libnvblas11
  libnvidia-compute-495 libnvidia-compute-510 libnvidia-compute-525 libnvidia-ml-dev libnvjpeg11
  libnvrtc-builtins11.5 libnvrtc11.2 libnvtoolsext1 libnvvm4 libopengl-dev libr

In [3]:
!nvidia-smi

Fri Oct 27 13:38:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [5]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-thmmscit
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-thmmscit
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=cef17305e44c1ba65a138afd6f90f181845e34925d821a3c1cac306bac674eda
  Stored in directory: /tmp/pip-ephem-wheel-cache-82vrtq14/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [6]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [37]:
%%cu
#include <stdio.h>
#include <time.h>
#include <thrust/extrema.h>
#include <thrust/device_vector.h>


#define CSC(call) 							\
do { 										\
	cudaError_t status = call;				\
	if (status != cudaSuccess) {																				\
		fprintf(stderr, "ERROR is %s:%d. Message: %s\n", __FILE__, __LINE__, cudaGetErrorString(status));		\
		exit(0);								\
	}											\
} while(0)


struct comparator {
	__host__ __device__ bool operator()(double a, double b) {		// Функция которая сравнивает объекты на "<"
		return abs(a) < abs(b); 									// operator() - переопределение оператора "()" для экземпляра этой структуры
	}
};


__global__ void gauss_method(double* m, int n, int cur_row) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    int idy = blockDim.y * blockIdx.y + threadIdx.y;
    int offsetx = blockDim.x * gridDim.x;
    int offsety = blockDim.y * gridDim.y;

    for (int i = cur_row + idx; i < n; i += offsetx) {
        for (int j = cur_row + idy; j < n + 1; j += offsety) {
            m[i + j * n] -= m[cur_row - 1 + j * n] / m[cur_row - 1 + (cur_row - 1) * n] * m[i + (cur_row - 1) * n];
        }
    }
}


__global__ void swap(double* m, int n, int row1, int row2) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    int offset = blockDim.x * gridDim.x;

    for (int i = idx; i < n + 1; i += offset) {
        double tmp = m[row1 + i * n];
        m[row1 + i * n] = m[row2 + i * n];
        m[row2 + i * n] = tmp;
    }
}


int main() {
    int n;
    comparator comp;
    n = 500;
    double* m = (double*)malloc(sizeof(double) * n * (n + 1));
    double* res = (double*)malloc(sizeof(double) * n);
	  for (int i = 0; i < n; i++) {
		    for (int j = 0; j < n; j++) {
			      m[i + j * n] = 1;
		    }
	  }
    for(int i = 0; i < n; i++) {
        m[i + n * n] = 1;
    }

    double* device_m;
    cudaMalloc(&device_m, sizeof(double) * n * (n + 1));
    cudaMemcpy(device_m, m, sizeof(double) * n * (n + 1), cudaMemcpyHostToDevice);

    thrust::device_ptr<double> m_ptr = thrust::device_pointer_cast(device_m);	// Трастовские функции принимают свой тип указателей, поэтому выполняем приведение типов.

    cudaEvent_t start, stop;
	  CSC(cudaEventCreate(&start));
	  CSC(cudaEventCreate(&stop));
	  CSC(cudaEventRecord(start));

    for (int i = 0; i < n; i++) {
        thrust::device_ptr<double> res = thrust::max_element(m_ptr + i * n + i, m_ptr + (i + 1) * n, comp);		// Ищем максимум в массиве на GPU
        int cur_i = (int)(res - (m_ptr + i * n));
        if (cur_i != i) {
            swap<<<128, 128>>>(device_m, n, cur_i, i);
            CSC(cudaDeviceSynchronize());
	          CSC(cudaGetLastError());
        }
        gauss_method<<<64, 64>>>(device_m, n, i + 1);
        CSC(cudaDeviceSynchronize());
	      CSC(cudaGetLastError());
    }

	  CSC(cudaEventRecord(stop));
	  CSC(cudaEventSynchronize(stop));
	  float t;
	  CSC(cudaEventElapsedTime(&t, start, stop));
	  CSC(cudaEventDestroy(start));
	  CSC(cudaEventDestroy(stop));

	  printf("time = %f ms\n", t);

    cudaMemcpy(m, device_m, sizeof(double) * n * (n + 1), cudaMemcpyDeviceToHost);

    for (int i = n - 1; i >= 0; i--) {
		    res[i] = m[i + n * n];
		    for (int j = i + 1; j < n; j++) {
			      res[i] -= res[j] * m[i + j * n];
		    }
		    res[i] /= m[i + i * n];
	  }

    for (int i = 0; i < n; i++) {
        printf("%.10lf ", res[i]);
    }

    free(m);
    free(res);
    cudaFree(device_m);
    return 0;
}

time = 339.031586 ms
-nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan -nan

In [17]:
!nvcc prof.cu

In [21]:
!ncu --metrics l1tex__data_bank_conflicts_pipe_lsu_mem_shared_op_ld,l1tex__data_bank_conflicts_pipe_lsu_mem_shared_op_st,l1tex__t_sectors_pipe_lsu_mem_global_op_ld,l1tex__t_sectors_pipe_lsu_mem_global_op_st,smsp__branch_targets_threads_divergent,sm__sass_inst_executed_op_local ./a.out

==PROF== Connected to process 12536 (/content/a.out)
==PROF== Profiling "_kernel_agent" - 0: 0%....50%....100% - 5 passes
==PROF== Profiling "_kernel_agent" - 1: 0%....50%....100% - 5 passes
==PROF== Profiling "_kernel_agent" - 2: 0%....50%....100% - 5 passes
==PROF== Profiling "gauss_method" - 3: 0%....50%....100% - 5 passes
==PROF== Profiling "_kernel_agent" - 4: 0%....50%....100% - 5 passes
==PROF== Profiling "_kernel_agent" - 5: 0%....50%....100% - 5 passes
==PROF== Profiling "_kernel_agent" - 6: 0%....50%....100% - 3 passes

==ERROR== LaunchFailed

==ERROR== LaunchFailed
==PROF== Trying to shutdown target application
==ERROR== The application returned an error code (9).
==ERROR== An error occurred while trying to profile.
[12536] a.out@127.0.0.1
  void thrust::cuda_cub::core::_kernel_agent<thrust::cuda_cub::__reduce::ReduceAgent<thrust::zip_iterator<thrust::tuple<thrust::device_ptr<double>, thrust::cuda_cub::counting_iterator_t<long>, thrust::null_type, thrust::null_type, thrust::